# **Построение моделей временных рядов для средних ставок по кредитам**

**Сначала просто подгрузим файл и разберем на переменные**

In [ ]:
from google.colab import files
import pandas as pd
# подгружаем файл loans_ind
uploaded = files.upload()

Saving loans_ind.xlsx to loans_ind (3).xlsx


Считываем файл: ,берем 1-8 столбцы, начиная с 6 строки

In [ ]:
df = pd.read_excel(
    'loans_ind.xlsx',
    skiprows=5,                # пропускаем первые 5 строк
    skipfooter=1,
    usecols=[0, 1, 2, 3, 4, 5, 6, 7],  # столбцы 0-7 (A-H)
    header=None,
    names=['date', 'rate_30', 'rate_31_90', 'rate_91_180', 'rate_181_1','rate_1', 'rate_1_3', 'rate_3_']
)

Преобразуем типы данных столбцов

In [ ]:
numeric_columns = ['rate_30', 'rate_31_90', 'rate_91_180', 'rate_181_1','rate_1', 'rate_1_3', 'rate_3_']

for col in numeric_columns:
    df[col] = pd.to_numeric(
        df[col].astype(str).str.replace(',', '.'),
        errors='coerce'
    )

# Создаем список месяцев для преобразования
month_mapping = {
    'Январь': 'January', 'Февраль': 'February', 'Март': 'March',
    'Апрель': 'April', 'Май': 'May', 'Июнь': 'June',
    'Июль': 'July', 'Август': 'August', 'Сентябрь': 'September',
    'Октябрь': 'October', 'Ноябрь': 'November', 'Декабрь': 'December'
}

# Преобразуем русские названия месяцев в английские и создаем даты
def parse_custom_date(date_str):
    try:
        # Разделяем на месяц и год
        parts = date_str.strip().split()
        if len(parts) == 2:
            month_ru, year = parts
            month_en = month_mapping.get(month_ru, month_ru)
            # Создаем дату (первое число месяца)
            return pd.to_datetime(f'{year}-{month_en}', format='%Y-%B')
    except:
        return pd.NaT
    return pd.NaT

# Применяем функцию к столбцу с датами
df['date'] = df['date'].astype(str).apply(parse_custom_date)
df['date'] = df['date'].dt.to_period('M')

print("После преобразования типов:")
print(df.head())
print(f"\nТипы данных после преобразования: {df.dtypes}")

После преобразования типов:
      date  rate_30  rate_31_90  rate_91_180  rate_181_1  rate_1  rate_1_3  \
0  2014-01    28.04       16.09        30.39       23.81   24.41     21.69   
1  2014-02    28.25       15.35        23.72       22.58   22.77     20.99   
2  2014-03    26.37       15.09        25.27       23.67   23.78     20.76   
3  2014-04    23.52       15.99        23.17       20.58   20.97     20.84   
4  2014-05    27.48       15.79        26.56       24.90   24.89     20.83   

   rate_3_  
0    17.29  
1    17.21  
2    16.98  
3    16.96  
4    16.82  

Типы данных после преобразования: date           period[M]
rate_30          float64
rate_31_90       float64
rate_91_180      float64
rate_181_1       float64
rate_1           float64
rate_1_3         float64
rate_3_          float64
dtype: object


Создаем CSV из них

In [ ]:
df.to_csv('processed_data.csv', index=False, encoding='utf-8')

Этот файл можно скачать

In [ ]:
from google.colab import files
files.download('processed_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Проверка стационарности рядов

In [ ]:
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error

def check_stationarity(series):
    """Проверка стационарности временного ряда"""
    print(f"\n--- Проверка стационарности ---")

    # Тест Дики-Фуллера
    result = adfuller(series)
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')

    if result[1] <= 0.05:
        print("Ряд СТАЦИОНАРЕН (отвергаем H0)")
        return True
    else:
        print("Ряд НЕ СТАЦИОНАРЕН (не отвергаем H0)")
        return False

for col in numeric_columns:
    series = df[col].dropna()
    print(f"Анализируем столбец: {col}")
    is_stationary = check_stationarity(series)

Анализируем столбец: rate_30

--- Проверка стационарности ---
ADF Statistic: -0.9329
p-value: 0.7770
Ряд НЕ СТАЦИОНАРЕН (не отвергаем H0)
Анализируем столбец: rate_31_90

--- Проверка стационарности ---
ADF Statistic: 0.1030
p-value: 0.9663
Ряд НЕ СТАЦИОНАРЕН (не отвергаем H0)
Анализируем столбец: rate_91_180

--- Проверка стационарности ---
ADF Statistic: -0.8278
p-value: 0.8108
Ряд НЕ СТАЦИОНАРЕН (не отвергаем H0)
Анализируем столбец: rate_181_1

--- Проверка стационарности ---
ADF Statistic: -1.9020
p-value: 0.3311
Ряд НЕ СТАЦИОНАРЕН (не отвергаем H0)
Анализируем столбец: rate_1

--- Проверка стационарности ---
ADF Statistic: -1.5113
p-value: 0.5279
Ряд НЕ СТАЦИОНАРЕН (не отвергаем H0)
Анализируем столбец: rate_1_3

--- Проверка стационарности ---
ADF Statistic: -1.0789
p-value: 0.7234
Ряд НЕ СТАЦИОНАРЕН (не отвергаем H0)
Анализируем столбец: rate_3_

--- Проверка стационарности ---
ADF Statistic: -1.1368
p-value: 0.7002
Ряд НЕ СТАЦИОНАРЕН (не отвергаем H0)


Дифференцируем ряды (избавляемся от сезонности, чтобы ряд стал стационарным)

In [ ]:
def find_optimal_d(series, max_d=3):
    """Находит оптимальный порядок дифференцирования"""
    original_pvalue = adfuller(series)[1]

    if original_pvalue <= 0.05:
        print("Исходный ряд стационарен")
        return 0, series

    for d in range(1, max_d + 1):
        diff_series = series.diff(d).dropna()
        p_value = adfuller(diff_series)[1]

        print(f"d={d}: p-value = {p_value:.4f}")

        if p_value <= 0.05:
            print(f"Стационарность достигнута при d={d}")
            return d, diff_series

    print(" Не удалось достичь стационарности, используем d=1")
    return 1, series.diff().dropna()

# Применяем ко всем столбцам
for col in numeric_columns:
    series = df[col].dropna()
    print(f"\n--- {col} ---")
    optimal_d, stationary_series = find_optimal_d(series)
    print(f"Оптимальный порядок дифференцирования: d={optimal_d}")


--- rate_30 ---
d=1: p-value = 0.3765
d=2: p-value = 0.4174
d=3: p-value = 0.3934
 Не удалось достичь стационарности, используем d=1
Оптимальный порядок дифференцирования: d=1

--- rate_31_90 ---
d=1: p-value = 0.1154
d=2: p-value = 0.2967
d=3: p-value = 0.2850
 Не удалось достичь стационарности, используем d=1
Оптимальный порядок дифференцирования: d=1

--- rate_91_180 ---
d=1: p-value = 0.0000
Стационарность достигнута при d=1
Оптимальный порядок дифференцирования: d=1

--- rate_181_1 ---
d=1: p-value = 0.0000
Стационарность достигнута при d=1
Оптимальный порядок дифференцирования: d=1

--- rate_1 ---
d=1: p-value = 0.0000
Стационарность достигнута при d=1
Оптимальный порядок дифференцирования: d=1

--- rate_1_3 ---
d=1: p-value = 0.0000
Стационарность достигнута при d=1
Оптимальный порядок дифференцирования: d=1

--- rate_3_ ---
d=1: p-value = 0.0002
Стационарность достигнута при d=1
Оптимальный порядок дифференцирования: d=1


# **Строим модель ARIMA для каждого столбца.**
 Сначала разбиваем выборку на тест и трейн. Тестовая максимум 10, тк модели тяжело предсказывать значения на большее количество шагов вперед.

(1 - φ₁B - φ₂B² - ... - φₚBᵖ) × (1 - B)ᵈ × yₜ = (1 + θ₁B + θ₂B² + ... + θₚBᵖ) × εₜ

B - оператор лага: B×yₜ = yₜ₋₁

εₜ - белый шум (ошибка)

yₜ - значение ряда в момент t


**Параметры:**
1. AR(p) - Авторегрессия (p)
p - сколько прошлых значений использовать
2. I(d) - Интегрирование (d)
d - порядок дифференцирования. Преобразует нестационарный ряд в стационарный
3. MA(q) - Скользящее среднее (q)
q - сколько прошлых ошибок использовать

рассматриваем от 0 до 2, тк при больих возникает риск переобучения

AIC = 2k - 2ln(L)

k = количество параметров модели (p + q)

L = правдоподобие модели (likelihood)

ln(L) = логарифм правдоподобия (мера "насколько хорошо модель описывает данные")

 AIC балансирует точность и сложность:
 -2ln(L) → чем меньше, тем лучше модель описывает данные (штраф за плохое описание)
 2k → штраф за сложность (больше параметров = больше штраф)

 Берем модель с меньшим AIC

MAE (Mean Absolute Error) - Средняя абсолютная ошибка:
Формула: MAE = (1/n) × Σ|y_true - y_pred|

RMSE (Root Mean Square Error) - Скорень из средней квадратичной ошибки:
Формула: RMSE = √[(1/n) × Σ(y_true - y_pred)²]

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore') # скипаем предупреждения о неоптимальных параметрах
df_new=pd.DataFrame()

def build_arima_model(dates,series, column_name, forecast_steps):
    train_size = int(len(series)-forecast_steps)
    train = series[:train_size]
    test = series[train_size:]

    print(f"Train: {len(train)} значений, Test: {len(test)} значений")

    # Подбор параметров ARIMA
    best_order = None
    best_aic = np.inf
    best_model = None

    # Перебираем возможные параметры
    param_combinations = [(0,1,1), (0,1,2), (1,1,0), (1,1,1), (1,1,2),
    (2,1,0), (2,1,1), (2,1,2), (0,2,1), (1,2,1),
    (0,0,1), (1,0,0), (1,0,1), (2,0,2)]

    for order in param_combinations:
      try:
        model = ARIMA(train, order=order)
        fitted_model = model.fit()
        if fitted_model.aic < best_aic:
          best_aic = fitted_model.aic
          best_order = order
          best_model = fitted_model
      except Exception as e:
        continue


    if best_model is None:
        print("Не удалось построить ни одну модель, используем ARIMA(1,1,1) по умолчанию")
        best_order = (1,1,1)
        best_model = ARIMA(train, order=best_order).fit()
        best_aic = best_model.aic

    print(f" Лучшие параметры ARIMA{best_order}: AIC = {best_aic:.2f}")

    # Прогноз на тестовой выборке
    test_forecast = best_model.forecast(steps=len(test))
    for date, actual, forecast in zip(dates, test, test_forecast):
        print(f"   {date.strftime('%Y-%m')}: факт = {actual:.2f}, прогноз = {forecast:.2f}")

    # Метрики качества
    mae = mean_absolute_error(test, test_forecast)
    rmse = np.sqrt(mean_squared_error(test, test_forecast))

    print(f" Качество модели на тесте:")
    print(f"   MAE: {mae:.2f}")
    print(f"   RMSE: {rmse:.2f}")

    # Прогноз на будущее
    final_model = ARIMA(series, order=best_order).fit()
    future_forecast = final_model.forecast(steps=forecast_steps)

    # Создаем будущие даты с 1-ми числами
    last_date = dates.iloc[-1]
    year = last_date.year
    month = last_date.month

    future_dates = []
    for i in range(1, forecast_steps + 1):
        next_month = month + i
        next_year = year
        if next_month > 12:
            next_month -= 12
            next_year += 1
        future_date = pd.Timestamp(year=next_year, month=next_month, day=1)
        future_dates.append(future_date)

    print(f"Прогноз на НОВЫЕ периоды ({len(series)}-{len(series)+forecast_steps-1}):")
    for date, value in zip(future_dates, future_forecast):
      print(f"   {date.strftime('%Y-%m')}: {value:.2f}")
    results=[round(x,2) for x in future_forecast.tolist()]
    return future_dates,results


for col in numeric_columns:
    print('----------------------------------------',col,'-----------------------------------------------------')
    series = df[col].dropna()
    dates,res=build_arima_model(df['date'],series, col, forecast_steps=3)
    if col==numeric_columns[0]:
      df_new['date']=[date.strftime('%Y-%m') for date in dates]
    df_new[col]=res

---------------------------------------- rate_30 -----------------------------------------------------
Train: 137 значений, Test: 3 значений
 Лучшие параметры ARIMA(1, 1, 2): AIC = 759.31
   2014-01: факт = 36.38, прогноз = 43.62
   2014-02: факт = 29.58, прогноз = 44.31
   2014-03: факт = 42.01, прогноз = 44.89
 Качество модели на тесте:
   MAE: 8.28
   RMSE: 9.62
Прогноз на НОВЫЕ периоды (140-142):
   2025-09: 41.10
   2025-10: 40.23
   2025-11: 39.50
---------------------------------------- rate_31_90 -----------------------------------------------------
Train: 137 значений, Test: 3 значений
 Лучшие параметры ARIMA(2, 1, 1): AIC = 736.85
   2014-01: факт = 28.08, прогноз = 37.33
   2014-02: факт = 43.42, прогноз = 37.26
   2014-03: факт = 43.37, прогноз = 42.27
 Качество модели на тесте:
   MAE: 5.51
   RMSE: 6.45
Прогноз на НОВЫЕ периоды (140-142):
   2025-09: 35.21
   2025-10: 48.40
   2025-11: 34.12
---------------------------------------- rate_91_180 ----------------------------

Создаем и скачиваем CSV

In [ ]:
df_arima_3=pd.concat([df,df_new])

# Сохраняем в CSV
df_arima_3.to_csv('arima_3.csv', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('arima_3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Получили у нестационарных рядов большие ошибки. Попробуем построить прогноз на 6 месяцев

In [ ]:
df_new1=pd.DataFrame()
for col in numeric_columns:
    print('----------------------------------------',col,'-----------------------------------------------------')
    series = df[col].dropna()
    dates,res=build_arima_model(df['date'],series, col, forecast_steps=6)
    if col==numeric_columns[0]:
      df_new1['date']=[date.strftime('%Y-%m') for date in dates]
    df_new1[col]=res

---------------------------------------- rate_30 -----------------------------------------------------
Train: 134 значений, Test: 6 значений
 Лучшие параметры ARIMA(1, 1, 2): AIC = 744.98
   2014-01: факт = 42.06, прогноз = 40.34
   2014-02: факт = 43.72, прогноз = 41.40
   2014-03: факт = 40.73, прогноз = 42.29
   2014-04: факт = 36.38, прогноз = 43.02
   2014-05: факт = 29.58, прогноз = 43.63
   2014-06: факт = 42.01, прогноз = 44.14
 Качество модели на тесте:
   MAE: 4.74
   RMSE: 6.54
Прогноз на НОВЫЕ периоды (140-145):
   2025-09: 41.10
   2025-10: 40.23
   2025-11: 39.50
   2025-12: 38.89
   2026-01: 38.39
   2026-02: 37.97
---------------------------------------- rate_31_90 -----------------------------------------------------
Train: 134 значений, Test: 6 значений
 Лучшие параметры ARIMA(2, 1, 1): AIC = 722.60
   2014-01: факт = 43.65, прогноз = 46.43
   2014-02: факт = 32.15, прогноз = 32.48
   2014-03: факт = 43.10, прогноз = 42.22
   2014-04: факт = 28.08, прогноз = 38.57
   

Сохраняем файл CSV

In [96]:
df_arima_6=pd.concat([df,df_new1])

# Сохраняем в CSV
df_arima_6.to_csv('arima_6.csv', index=False, encoding='utf-8-sig')

files.download('arima_6.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Теперь построим модель SARIMAХ**
для прогнза на 3 месяца вперед

Φ(L)Φₛ(Lₛ)(1-L)ᵈ(1-Lₛ)ᴰyₜ = c + Θ(L)Θₛ(Lₛ)εₜ

**Операторы:**

L - лаговый оператор: L·yₜ = yₜ₋₁ (значение в предыдущий период)

Lₛ - сезонный лаговый оператор: Lₛ·yₜ = yₜ₋ₛ (значение сезонность назад)

s - период сезонности (12 для месячных данных, 4 для квартальных)

**Несезонная часть ARIMA(p,d,q):**
p - порядок авторегрессии (AR)

d - порядок дифференцирования (I)

q - порядок скользящего среднего (MA)

**Сезонная часть (P,D,Q,s):**
P - порядок сезонной авторегрессии (SAR)

D - порядок сезонного дифференцирования (SI)

Q - порядок сезонного скользящего среднего (SMA)

φ  - коэффициенты несезонной авторегрессии

Φ  - коэффициенты сезонной авторегрессии

θ  - коэффициенты несезонного скользящего среднего

Θ  - коэффициенты сезонного скользящего среднего

c - константа модели

ε  - ошибка (белый шум)

**Пример: SARIMA(1,1,1)(1,1,1,12)**

(1 - φL)(1 - ΦL¹²)(1-L)(1-L¹²)yₜ = c + (1 + θL)(1 + ΘL¹²)εₜ


In [98]:
import numpy as np
import warnings
warnings.filterwarnings('ignore') # скипаем предупреждения о неоптимальных параметрах

def build_sarima_model(dates, series, column_name, forecast_steps):
    train_size = int(len(series)-forecast_steps)
    train = series[:train_size]
    test = series[train_size:]

    print(f"Train: {len(train)} значений, Test: {len(test)} значений")

    # Подбор параметров SARIMA
    best_order = None
    best_seasonal_order = None
    best_aic = np.inf
    best_model = None

    # Перебираем возможные параметры
    param_combinations = [
    (0,1,1), (1,1,0), (1,1,1), (2,1,2), (0,1,2), (2,1,1),
    (0,2,1), (1,2,1), (2,2,2), (1,0,1), (2,0,2)
    ]

    seasonal_combinations = [
        (0,1,1,12), (1,1,0,12), (1,1,1,12), (0,0,1,12), (1,0,1,12),
        (0,1,0,12), (1,0,0,12), (2,1,1,12), (1,1,2,12), (0,0,0,12)
    ]

    for order in param_combinations:
        for seasonal_order in seasonal_combinations:
            try:
                model = SARIMAX(train, order=order, seasonal_order=seasonal_order,
                              enforce_stationarity=False, enforce_invertibility=False)
                fitted_model = model.fit(disp=False)
                if fitted_model.aic < best_aic:
                    best_aic = fitted_model.aic
                    best_order = order
                    best_seasonal_order = seasonal_order
                    best_model = fitted_model
            except Exception as e:
                continue

    if best_model is None:
        print("Не удалось построить ни одну модель, используем SARIMA(1,1,1)x(1,1,1,12) по умолчанию")
        best_order = (1,1,1)
        best_seasonal_order = (1,1,1,12)
        model = SARIMAX(train, order=best_order, seasonal_order=best_seasonal_order,
                      enforce_stationarity=False, enforce_invertibility=False)
        best_model = model.fit(disp=False)
        best_aic = best_model.aic

    print(f" Лучшие параметры SARIMA{best_order}x{best_seasonal_order}: AIC = {best_aic:.2f}")

    # Прогноз на тестовой выборке
    test_forecast = best_model.forecast(steps=len(test))
    for date, actual, forecast in zip(dates, test, test_forecast):
        print(f"   {date.strftime('%Y-%m')}: факт = {actual:.2f}, прогноз = {forecast:.2f}")

    # Метрики качества
    mae = mean_absolute_error(test, test_forecast)
    rmse = np.sqrt(mean_squared_error(test, test_forecast))

    print(f" Качество модели на тесте:")
    print(f"   MAE: {mae:.2f}")
    print(f"   RMSE: {rmse:.2f}")

    # Прогноз на будущее
    final_model = SARIMAX(series, order=best_order, seasonal_order=best_seasonal_order,
                        enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
    future_forecast = final_model.forecast(steps=forecast_steps)

    # Создаем будущие даты с 1-ми числами
    last_date = dates.iloc[-1]
    year = last_date.year
    month = last_date.month

    future_dates = []
    for i in range(1, forecast_steps + 1):
        next_month = month + i
        next_year = year
        if next_month > 12:
            next_month -= 12
            next_year += 1
        future_date = pd.Timestamp(year=next_year, month=next_month, day=1)
        future_dates.append(future_date)

    print(f"Прогноз на НОВЫЕ периоды ({len(series)}-{len(series)+forecast_steps-1}):")
    for date, value in zip(future_dates, future_forecast):
      print(f"   {date.strftime('%Y-%m')}: {value:.2f}")
    results=[round(x,2) for x in future_forecast.tolist()]
    return future_dates,results

df_new2=pd.DataFrame()
for col in numeric_columns:
    print('----------------------------------------',col,'-----------------------------------------------------')
    series = df[col].dropna()
    dates,res=build_sarima_model(df['date'],series, col, forecast_steps=3)
    if col==numeric_columns[0]:
      df_new2['date']=[date.strftime('%Y-%m') for date in dates]
    df_new2[col]=res

---------------------------------------- rate_30 -----------------------------------------------------
Train: 137 значений, Test: 3 значений
 Лучшие параметры SARIMA(2, 1, 2)x(1, 1, 2, 12): AIC = 512.51
   2014-01: факт = 36.38, прогноз = 45.67
   2014-02: факт = 29.58, прогноз = 29.60
   2014-03: факт = 42.01, прогноз = 42.34
 Качество модели на тесте:
   MAE: 3.21
   RMSE: 5.36
Прогноз на НОВЫЕ периоды (140-142):
   2025-09: 47.01
   2025-10: 42.22
   2025-11: 47.65
---------------------------------------- rate_31_90 -----------------------------------------------------
Train: 137 значений, Test: 3 значений
 Лучшие параметры SARIMA(0, 1, 2)x(1, 1, 2, 12): AIC = 512.25
   2014-01: факт = 28.08, прогноз = 32.39
   2014-02: факт = 43.42, прогноз = 47.92
   2014-03: факт = 43.37, прогноз = 50.18
 Качество модели на тесте:
   MAE: 5.21
   RMSE: 5.33
Прогноз на НОВЫЕ периоды (140-142):
   2025-09: 29.02
   2025-10: 43.08
   2025-11: 29.73
---------------------------------------- rate_91_18

Сохраняем файл CSV

In [99]:
df_sarima_3=pd.concat([df,df_new2])

# Сохраняем в CSV
df_sarima_3.to_csv('sarima_3.csv', index=False, encoding='utf-8-sig')

files.download('sarima_3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SARIMA для прогноза на 6 месяцев:

In [ ]:
df_new3=pd.DataFrame()
for col in numeric_columns:
    print('----------------------------------------',col,'-----------------------------------------------------')
    series = df[col].dropna()
    dates,res=build_sarima_model(df['date'],series, col, forecast_steps=6)
    if col==numeric_columns[0]:
      df_new3['date']=[date.strftime('%Y-%m') for date in dates]
    df_new3[col]=res

---------------------------------------- rate_30 -----------------------------------------------------
Train: 134 значений, Test: 6 значений
 Лучшие параметры SARIMA(2, 1, 2)x(1, 1, 2, 12): AIC = 498.81
   2014-01: факт = 42.06, прогноз = 43.88
   2014-02: факт = 43.72, прогноз = 47.18
   2014-03: факт = 40.73, прогноз = 43.55
   2014-04: факт = 36.38, прогноз = 48.04
   2014-05: факт = 29.58, прогноз = 32.07
   2014-06: факт = 42.01, прогноз = 44.63
 Качество модели на тесте:
   MAE: 4.15
   RMSE: 5.36
Прогноз на НОВЫЕ периоды (140-145):
   2025-09: 47.01
   2025-10: 42.22
   2025-11: 47.65
   2025-12: 38.35
   2026-01: 47.50
   2026-02: 49.46
---------------------------------------- rate_31_90 -----------------------------------------------------
Train: 134 значений, Test: 6 значений
 Лучшие параметры SARIMA(0, 1, 2)x(1, 1, 2, 12): AIC = 496.15
   2014-01: факт = 43.65, прогноз = 45.95
   2014-02: факт = 32.15, прогноз = 28.12
   2014-03: факт = 43.10, прогноз = 48.12
   2014-04: фак

Сохраняем файл CSV

In [101]:
df_sarima_6=pd.concat([df,df_new3])

# Сохраняем в CSV
df_sarima_6.to_csv('sarima_6.csv', index=False, encoding='utf-8-sig')

files.download('sarima_6.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>